In [1]:
import os, sys

parentdir = os.path.abspath('/home/xcheng/mvesc/ETL')
sys.path.insert(0,parentdir)
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Image
from IPython.display import display
from numpy import random
import pickle
from estimate_prediction_model import *
from write_to_database import write_scores_to_db
from optparse import OptionParser
import re
%load_ext autotime
%matplotlib inline

In [2]:
tab_reports = 'model.reports' # it has batch_name, precision, recall
tab_feature_scores = 'model.feature_scores'
tab_prediction = 'model.predictions'
dir_pkls = '/mnt/data/mvesc/Models_Results/pkls'
pkls_logit = ['08_12_2016_grade_7_param_set_17_logit_ht_19082']
pkls_logit = os.path.join(dir_pkls, pkls_logit[0])+'_logit.pkl'
with open(pkls_logit, 'rb') as handle:
    pkl = pickle.load(handle)
    
with open(dir_pkls+'/08_12_2016_grade_7_param_set_17_logit_ht_19078_logit.pkl', 'rb') as handle:
    pkl2 = pickle.load(handle)
    

time: 59 ms


In [3]:
pkl['']

{'cross_validation_scores': array([ 0.24293077,  0.2596474 ]),
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'features': Index(['num_stem_classes_gr_5', 'num_future_prep_classes_gr_6',
        'num_health_classes_gr_5', 'percent_passed_pf_classes_gr_5', 'iss_gr_5',
        'avg_address_change_to_gr_5', 'n_districts_to_gr_5',
        'n_records_to_gr_5', 'avg_district_change_to_gr_5',
        'n_addresses_to_gr_5',
        ...
        'n_records_to_gr_6', 'num_humanities_classes_gr_6',
        'avg_city_change_to_gr_6', 'oss_gr_5', 'math_normalized_gr_5',
        'num_pf_classes_gr_5', 'num_art_classes_gr_5', 'gpa_gr_6',
        'stem_gpa_gr_5', 'num_health_classes_gr_6'],
       dtype='object', length=188),
 'file_name': '08_12_2016_grade_7_param_set_1

time: 101 ms


In [12]:
int(pkl['model_options']['cohort_grade_level_begin'].split('_')[-1][:-2])

7

time: 2.52 ms


'11'

time: 2.29 ms


In [3]:
# it seems the robusted-scaled are centered to median and rescaled by IQR;
# we may assume all features has a simiar scales

"""
Procedures to generate individual scores and top risk factors of logistic regression
1. load the features of current students at a certain grade;
2. load the corresonding model pickle file;
3. make predictions to get raw score (0, 1)
4. rank the scores and assiged risk level and risk score (5% high risk, 5% medium, 5% low; rescale 20% to 1 to 10);
5. find the top risk factors/features/column names;
6. save as pdf output to csv
"""
schema, table = 'model', 'individual_risk_scores_factors'
dir_pkls = '/mnt/data/mvesc/Models_Results/pkls'
pickle_file = '08_12_2016_grade_7_param_set_17_logit_ht_19082_logit.pkl'
if_exists = 'replace'
random_seed = 62571
topK = 3
both_positive_negative = True
num_students = 20
student_column = 'student_lookup'
pkl_model_key = 'estimator'

def topK_features_logit(model, data, feature_name, topN=3):
    importances = np.transpose(model.coef_)[:, 0]*data
    indices = importances.argsort()
    indices = indices[::-1]
    #print(indices[:3])
    return(list(np.array(features)[indices[:3]]))

with open(os.path.join(dir_pkls, pickle_file), 'rb') as handle:
    pkl = pickle.load(handle)
features = list(pkl['features']) # to pull feature data later

###!!! generate random all_features_dataframe with student_lookups for testing 
np.random.seed(random_seed)
all_data4prediction = np.random.rand(num_students, len(features))
all_data4prediction = pd.DataFrame(all_data4prediction, columns=features)
all_data4prediction[student_column] = range(1, all_data4prediction.shape[0]+1)
all_data4prediction = all_data4prediction[[student_column]+features]
###!!! random all_features_dataframe generated

student_lookups = all_data4prediction[student_column]
all_feature_data = all_data4prediction[features]
# all_feature_data = Robust_Scale(all_feature_data) #processed
risk_probas = pkl[pkl_model_key].predict_proba(all_feature_data)[:,1]
predictions = pkl[pkl_model_key].predict(all_feature_data)
top_individual_features = []
for i in range(all_feature_data.shape[0]):
    x = np.array(all_feature_data.iloc[i, :])
    top_individual_features.append(topN_features_logit(pkl[pkl_model_key], x, features, topN=3))

top_risk_factor_names = ['risk_factor_'+str(i) for i in range(1, topK+1)]
top_individual_features = pd.DataFrame(top_individual_features, columns=top_risk_factor_names)

# individual risk score & factors
individual_scores_factors = pd.DataFrame()
individual_scores_factors[student_column] = student_lookups
individual_scores_factors['risk_score'] =  risk_probas
individual_scores_factors = pd.concat([individual_scores_factors, top_individual_features], axis=1)

# individual risk facotrs values
top_feature_values = {'risk_factor_'+str(i):[] for i in range(1, topK+1)}
for risk_i in top_feature_values:
    for student_i in range(all_data4prediction.shape[0]):
        column_in_alldata = individual_scores_factors.ix[student_i, risk_i]
        top_feature_values[risk_i].append(all_data4prediction.ix[student_i, column_in_alldata])
top_feature_values = pd.DataFrame(top_feature_values)
top_feature_values = top_feature_values.rename(columns={x:x+'_value' for x in top_feature_values.columns})
individual_scores_factors = pd.concat([individual_scores_factors, top_feature_values], axis=1)

# model and its file name
individual_scores_factors['model'] = str(pkl[pkl_model_key])
individual_scores_factors['model_file'] = pickle_file

eng = postgres_engine_generator()
individual_scores_factors.to_sql(table, eng, schema = schema, if_exists=if_exists, index=False)

time: 3.37 s


In [4]:
individual_scores_factors.head()

,student_lookup,risk_score,risk_factor_1,risk_factor_2,risk_factor_3,risk_factor_1_value,risk_factor_2_value,risk_factor_3_value,model,model_file
0,1,0.758139,ethnicity_M,district_gr_6_Maysville,read_normalized_gr_5_isnull,0.722477,0.848921,0.876151,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
1,2,0.213100,ethnicity_M,disadvantagement_gr_6_economic,art_gpa_gr_6,0.416974,0.965883,0.760293,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
2,3,0.493108,ethnicity_M,art_gpa_gr_6,disadvantagement_gr_6_economic,0.654893,0.943058,0.789732,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
3,4,0.516004,ethnicity_M,district_gr_6_Maysville,read_normalized_gr_5_isnull,0.878060,0.996202,0.766150,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...
4,5,0.792288,ethnicity_M,district_gr_6_Maysville,art_gpa_gr_6,0.900475,0.863978,0.803051,"LogisticRegression(C=1.0, class_weight=None, d...",08_12_2016_grade_7_param_set_17_logit_ht_19082...


time: 18.6 ms


In [41]:
def read_in_model(filename, model_name,
        pkl_dir = '/mnt/data/mvesc/Models_Results/pkls'):
    full_filename = filename +'_' + model_name + '.pkl'
    with open(os.path.join(pkl_dir, full_filename), 'rb') as model:
        model_pkl = pickle.load(model)
    clf, options = model_pkl['estimator'], model_pkl['model_options']
    grade = int(model_pkl['model_options']['cohort_grade_level_begin'].split('_')[-1][:-2])
    return clf, options, grade

def build_test_feature_set(options, current_year = 2016):
    # get student list of 2016 students in specified cohort grade level
    with postgres_pgconnection_generator() as connection:
        cohort = options['cohort_grade_level_begin']
        test_outcomes = read_table_to_df(connection,
            table_name = 'outcome', schema = 'model', nrows = -1,
            columns = ['student_lookup', 'current_students', cohort])
        test_outcomes.dropna(subset=['current_students', cohort], inplace=True)
        test_outcomes = pd.DataFrame(test_outcomes.student_lookup[
            test_outcomes[cohort] == current_year])

        for table, column_names in options['features_included'].items():
            features = read_table_to_df(connection, table_name = table,
                schema = 'model', nrows = -1,
                columns=(['student_lookup'] + column_names))
            # join to only keep features for current_students
            test_outcomes = pd.merge(test_outcomes, features,
                how = 'left', on = 'student_lookup')

    # build dataframe containing student_lookup
    # and all features as numeric non-categorical values
    test_outcomes.set_index('student_lookup', inplace=True)
    test_outcomes_raw = test_outcomes
    test_outcomes = df2num(test_outcomes_raw, drop_reference = False,
        dummify = True, drop_entirely_null = False)
    return test_outcomes_raw, test_outcomes

def test_impute_and_scale(test_outcomes, options):
    all_past_data = build_outcomes_plus_features(options)
    train, val, val = temporal_cohort_test_split(all_past_data,
            options['cohort_grade_level_begin'],
            options['cohorts_test'], options['cohorts_val'],
            options['cohorts_training'])
    exclude = set((options['outcome_name'],
                options['cohort_grade_level_begin']))
    train = train.drop([options['outcome_name'],
            options['cohort_grade_level_begin']],axis=1)
    val = val.drop([options['outcome_name'],
            options['cohort_grade_level_begin']],axis=1)

    category_missing = [col for col in train.columns if
                    col not in test_outcomes.columns]
    for col in category_missing:
        test_outcomes[col] = 0
    test_outcomes = test_outcomes.filter(train.columns)

    # imputation for missing values in features
    train, val, test_outcomes = impute_missing_values(train, val, test_outcomes,
        options['missing_impute_strategy'])

    # feature scaling
    train, val, test_outcomes = scale_features(train, val, test_outcomes,
        options['feature_scaling'])

    assert (all(train.columns == test_outcomes.columns)),\
        "train and current_students have different columns"
    return test_outcomes

def make_and_save_predictions(future_predictions, clf, filename):
    # generate soft predictions
    if hasattr(clf, "predict_proba"):
        future_set_scores = clf.predict_proba(future_predictions)[:,1]
    else:
        future_set_scores = clf.decision_function(future_predictions)

    saved_outputs = {
        'file_name' : filename,
        'future_index' : future_predictions.index,
        'future_scores' : future_set_scores,
        'future_preds' : clf.predict(future_predictions)
    }
    #write_scores_to_db(saved_outputs, importance_scores = False)
    return(saved_outputs)

def topK_features_logit(model, data, feature_names, topK=3):
    importances = np.transpose(model.coef_)[:, 0]*data
    indices = importances.argsort()
    indices = indices[::-1]
    #print(indices[:3])
    return(list(np.array(feature_names)[indices[:topK]]))

def risk_score2level(score, percentiles, risk_levels):
    ind = (percentiles>score).sum()
    return(risk_levels[ind])

def get_school_district(df, grade, year=2015):
    with postgres_pgconnection_generator() as conn:
        with conn.cursor() as cursor:
            select_current_grade = """
            select student_lookup, grade, school_year, school_code, district
            from clean.all_snapshots
            where grade={g} and school_year={yr}
            """.format(g=grade-1, yr=2015)
            df_school_etc = pd.read_sql_query(select_current_grade, conn)
    return df.merge(df_school_etc, on='student_lookup')

def main():
    filename_list = ['08_12_2016_grade_6_param_set_8_logit_ht_16587', 
                     '08_12_2016_grade_7_param_set_17_logit_ht_19082', 
                     '08_12_2016_grade_8_param_set_16_logit_ht_19857', 
                     '08_12_2016_grade_9_param_set_22_logit_ht_22737',
                     '08_12_2016_grade_10_param_set_22_logit_ht_23459']
    topK = 3
    schema, table = 'model', 'individual_risk_scores_factors'
    dir_pkls = '/mnt/data/mvesc/Models_Results/pkls'
    if_exists = 'append'
    random_seed = 62571

    #if options.filename_list:
    #    filename_list = options.filename_list

    for filename in filename_list:
        # load saved model
        print("- Processing pkl: ", filename)
        model_name = filename.split('_')[-3]
        clf, options, grade = read_in_model(filename, model_name)

        # fetch and process feature data
        features_raw, feaures_num = build_test_feature_set(options)
        features_processed = test_impute_and_scale(feaures_num, options)

        # predict and find top factors
        risk_probas = clf.predict_proba(features_processed)[:,1]
        predictions = clf.predict(features_processed)
        top_individual_features = []
        for i in range(features_processed.shape[0]):
            x = np.array(features_processed.iloc[i, :])
            top_individual_features.append(topK_features_logit(clf, x, features_processed.columns, topK=topK))

        top_risk_factor_names = ['risk_factor_'+str(i) for i in range(1, topK+1)]
        top_individual_features = pd.DataFrame(top_individual_features, 
                                               columns=top_risk_factor_names)

        # individual risk score, level & factors
        individual_scores_factors = pd.DataFrame()
        individual_scores_factors['student_lookup'] = features_raw.index

        # assign risk score & levels
        individual_scores_factors['risk_score'] =  risk_probas
        percentiles = individual_scores_factors.risk_score.quantile(q=[0.95, 0.85, 0.70])
        risk_levels = ['High', 'Medium', 'Low', 'Safe']
        student_risk_levels = [risk_score2level(s, percentiles, risk_levels) for s in individual_scores_factors.risk_score]
        individual_scores_factors['risk_level'] = student_risk_levels
        individual_scores_factors = pd.concat([individual_scores_factors, top_individual_features], axis=1)

        top_feature_values = {'risk_factor_'+str(i):[] for i in range(1, topK+1)}
        for risk_i in top_feature_values:
            for student_i in range(features_processed.shape[0]):
                column_in_features_processed = individual_scores_factors.ix[student_i, risk_i]
                top_feature_values[risk_i].append(features_processed[column_in_features_processed].iloc[student_i])
        top_feature_values = pd.DataFrame(top_feature_values)
        top_feature_values = top_feature_values.rename(columns={x:x+'_value' for x in top_feature_values.columns})
        individual_scores_factors = pd.concat([individual_scores_factors, top_feature_values], axis=1)

        # subset the data to only include current students and corrent grades
        individual_scores_factors = get_school_district(individual_scores_factors, grade)

        # model and its file name
        individual_scores_factors['model'] = model_name
        individual_scores_factors['model_file'] = filename
        individual_scores_factors.sort_values(by=['risk_score'],inplace=True, ascending=False)


        # output to postgres
        eng = postgres_engine_generator()
        individual_scores_factors.to_sql(table, eng, schema = schema, if_exists=if_exists, index=False)
        print('- Processed ', filename)

- Processing pkl:  08_12_2016_grade_6_param_set_8_logit_ht_16587
- Processed  08_12_2016_grade_6_param_set_8_logit_ht_16587
- Processing pkl:  08_12_2016_grade_7_param_set_17_logit_ht_19082
- Processed  08_12_2016_grade_7_param_set_17_logit_ht_19082
- Processing pkl:  08_12_2016_grade_8_param_set_16_logit_ht_19857
- Processed  08_12_2016_grade_8_param_set_16_logit_ht_19857
- Processing pkl:  08_12_2016_grade_9_param_set_22_logit_ht_22737
- Processed  08_12_2016_grade_9_param_set_22_logit_ht_22737
- Processing pkl:  08_12_2016_grade_10_param_set_22_logit_ht_23459


ValueError: X has 85 features per sample; expecting 86

time: 29.7 s


In [37]:
individual_scores_factors.head()

,student_lookup,risk_score,risk_level,risk_factor_1,risk_factor_2,risk_factor_3,risk_factor_1_value,risk_factor_2_value,risk_factor_3_value,grade,school_code,district,model,model_file
1174,23779.0,1.000000,High,discipline_incidents_gr_6,art_gpa_gr_5,humanities_gpa_gr_5,21.0,-4.00,-2.60,6,8,Logan_Hocking,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1911,27633.0,0.999906,High,discipline_incidents_gr_6,humanities_gpa_gr_5,stem_gpa_gr_5,9.0,-2.60,-2.00,6,8,Logan_Hocking,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1233,23991.0,0.999836,High,discipline_incidents_gr_6,humanities_gpa_gr_5,stem_gpa_gr_5,10.0,-1.95,-1.15,6,8,Logan_Hocking,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1197,23945.0,0.999737,High,discipline_incidents_gr_6,humanities_gpa_gr_5,num_pf_classes_gr_6,10.0,-1.95,1.00,6,8,Logan_Hocking,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082
1775,26101.0,0.999229,High,discipline_incidents_gr_6,humanities_gpa_gr_5,stem_gpa_gr_5,9.0,-1.25,-1.00,6,8,Logan_Hocking,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082


time: 22.2 ms


time: 2.77 ms


In [33]:
get_school_district(individual_scores_factors, grade)

,student_lookup,risk_score,risk_level,risk_factor_1,risk_factor_2,risk_factor_3,risk_factor_1_value,risk_factor_2_value,risk_factor_3_value,model,model_file,grade,school_code,district
0,23779.0,1.000000,High,discipline_incidents_gr_6,art_gpa_gr_5,humanities_gpa_gr_5,21.000000,-4.000000,-2.600000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
1,27633.0,0.999906,High,discipline_incidents_gr_6,humanities_gpa_gr_5,stem_gpa_gr_5,9.000000,-2.600000,-2.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
2,23991.0,0.999836,High,discipline_incidents_gr_6,humanities_gpa_gr_5,stem_gpa_gr_5,10.000000,-1.950000,-1.150000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
3,23945.0,0.999737,High,discipline_incidents_gr_6,humanities_gpa_gr_5,num_pf_classes_gr_6,10.000000,-1.950000,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
4,26101.0,0.999229,High,discipline_incidents_gr_6,humanities_gpa_gr_5,stem_gpa_gr_5,9.000000,-1.250000,-1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
5,13240.0,0.999032,High,discipline_incidents_gr_6,humanities_gpa_gr_5,iss_gr_6,3.000000,-2.250000,5.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,MAMS,Maysville
6,25973.0,0.998392,High,discipline_incidents_gr_6,humanities_gpa_gr_5,num_pf_classes_gr_6,8.000000,-1.400000,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
7,24084.0,0.998275,High,discipline_incidents_gr_6,stem_gpa_gr_5,humanities_gpa_gr_5,9.000000,-1.000000,-0.550000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
8,25723.0,0.998094,High,discipline_incidents_gr_6,read_normalized_gr_4,num_pf_classes_gr_6,9.000000,-0.682836,1.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking
9,23955.0,0.998005,High,discipline_incidents_gr_6,humanities_gpa_gr_5,oss_gr_5,5.000000,-1.750000,3.000000,logit,08_12_2016_grade_7_param_set_17_logit_ht_19082,6,8,Logan_Hocking


time: 443 ms


In [28]:
inddf.columns

Index(['student_lookup', 'risk_score', 'risk_level', 'risk_factor_1',
       'risk_factor_2', 'risk_factor_3', 'risk_factor_1_value',
       'risk_factor_2_value', 'risk_factor_3_value', 'model', 'model_file',
       'grade', 'school_code', 'district'],
      dtype='object')

time: 2.46 ms
